In [ ]:
import json
import csv

inputfile = "/home/sx/MPI_profile/is.A.x_pmu/graph_test"
outputfile_featrue ="/home/sx/MPI_profile/is.A.x_pmu/graph_test_feature.csv"
outputfile_featrue_merged ="/home/sx/MPI_profile/is.A.x_pmu/graph_test_feature_merged.csv"
outputfile_edge = "/home/sx/MPI_profile/is.A.x_pmu/graph_test_edge.csv"

fieldnames = ['ts_id', 'tid', 'name', 'ts', 'dur', 'args_Data_type', 'args_Dest', 'args_Tag', 'args_Count',
            'args_Send_Count', 'args_Recv_Count', 'args_Comm', 'args_Request', 'args_Op', 'args_Root', 
            'arg_pmu_0', 'arg_pmu_1', 'arg_pmu_2', 'arg_pmu_3']

with open(inputfile, 'r') as file:
    data = json.load(file)

In [ ]:
with open(outputfile_featrue, 'w', newline='') as csvfile:
    feature = []
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for item in data['traceEvents']:
        if item['name'] == 'dep_flow_':
            break
        if 'dur' not in item:
            item['dur'] = 0
        if 'args' not in item:
            item['args'] = {}
        # print(item)
        if 'Data type' not in item['args']:
            item['args']['Data type'] = 0
        if 'Dest' not in item['args']:
            item['args']['Dest'] = 0
        if 'Tag' not in item['args']:
            item['args']['Tag'] = 0
        if 'Count' not in item['args']:
            item['args']['Count'] = 0
        if 'Comm' not in item['args']:
            item['args']['Comm'] = 0
        if 'Request' not in item['args']:
            item['args']['Request'] = 0
        if 'Op' not in item['args']:
            item['args']['Op'] = 0
        if 'Root' not in item['args']:
            item['args']['Root'] = 0
        if 'Backtrace' not in item['args']:
            item['args']['Backtrace'] = 0
        if 'Send Count' not in item['args']:
            item['args']['Send Count'] = 0
        if 'Recv Count' not in item['args']:
            item['args']['Recv Count'] = 0
        # print(item)
        row = {
            'ts_id': item['ts_id'],
            'tid': item['tid'],
            'name': item['name'],
            # 'cat': item['cat'],
            # 'ph': item['ph'],
            'ts': item['ts'],
            'dur': item['dur'],
            'args_Data_type': item['args']['Data type'],
            'args_Dest': item['args']['Dest'],
            'args_Tag': item['args']['Tag'],
            'args_Count': item['args']['Count'],
            'args_Send_Count': item['args']['Send Count'],
            'args_Recv_Count': item['args']['Recv Count'],
            'args_Comm': item['args']['Comm'],
            'args_Request': item['args']['Request'],
            'args_Op': item['args']['Op'],
            'args_Root': item['args']['Root'],
            # 'args_Backtrace': item['args']['Backtrace'],
            'arg_pmu_0': item['args']['PMU 0(PAPI_RES_STL) count'],
            'arg_pmu_1': item['args']['PMU 1(PAPI_LD_INS) count'],
            'arg_pmu_2': item['args']['PMU 2(PAPI_SR_INS) count'],
            'arg_pmu_3': item['args']['PMU 3(PAPI_L2_DCR) count'],
        }
        feature.append(row)
        print(row)
        # writer.writerow(row)

In [ ]:
import pandas as pd

df_feature = pd.DataFrame(feature)
df_feature_merged = df_feature.groupby(['ts_id', 'tid']).agg(lambda x: ','.join(x.astype(str))).reset_index()

print(df_feature_merged)
# df_feature_merged.to_csv(outputfile_featrue_merged, header=True, index=False)

In [ ]:
from gensim.models import Word2Vec
import pandas as pd
import os

dimension = 32

input_path = "/home/sx/MPI_profile/is.A.x_pmu/graph_test_feature_merged.csv"
output_path = "/home/sx/MPI_profile/is.A.x_pmu/graph_test_feature_vector.csv"

data = pd.read_csv(input_path, header=0)
sentences = data.reset_index(drop=False).apply(lambda row: row.tolist(), axis=1).tolist()
model = Word2Vec(sentences, vector_size=dimension, window=5, min_count=1, workers=4)

def get_vector(row):
    vectors = [model.wv[str(num)] for num in row if str(num) in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)  
    # else:
    #     return [0] * dimension  

vectors = [get_vector(row) for row in sentences]

df_vectors = pd.DataFrame(vectors, columns=[f'vector_{i}' for i in range(dimension)])
df_vectors = pd.concat([data.iloc[:, 0], data.iloc[:, 1], df_vectors], axis=1)

print(df_vectors)


In [ ]:
num = 8

def filter_tid(group, tid):
    filtered = group[group['tid'] == tid]
    if filtered.empty:
        filtered = pd.DataFrame({'ts_id': group.name, 'tid': [tid]})
        for i in range(32):
           filtered[f'vector_{i}'] = [0]
    return filtered

results = []
for tid in range(num):
    df_vectors_i = df_vectors.groupby('ts_id').apply(filter_tid, tid)
    df_vectors_i = df_vectors_i.reset_index(drop=True)
    results.append(df_vectors_i.iloc[:, 2:])
    df_combined = pd.concat(results, axis=1)
    df_combined = df_combined.reset_index(drop=True)


print(df_combined)

In [171]:
import itertools

dimension = 32
num = 8

nodes = [f'node_{i}' for i in range(num)]
vectors = [f'vector_{i}' for i in range(dimension)]

repeated_nodes = list(itertools.repeat(node, dimension) for node in nodes)
repeated_vectors = list(itertools.repeat(vectors, num))

nodes = [item for sublist in repeated_nodes for item in sublist]
vectors = [item for sublist in repeated_vectors for item in sublist]

df_metric = pd.DataFrame(data=[nodes], columns=vectors)
# print(df_metric)

df_result = pd.concat([df_metric, df_combined], ignore_index=True)

print(df_result)
# df_result.to_csv('/home/sx/MPI_profile/is.A.x_pmu/graph_test_metric.csv', header=True, index=False)


    vector_0  vector_1  vector_2  vector_3  vector_4  vector_5  vector_6  \
0     node_0    node_0    node_0    node_0    node_0    node_0    node_0   
1        0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2        0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3        0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4        0.0       0.0       0.0       0.0       0.0       0.0       0.0   
5   0.016772 -0.019168 -0.001217  0.001976  0.016888 -0.013172  0.007373   
6   -0.00366  0.002075  0.002881  0.009914 -0.001868  0.001454  0.011866   
7  -0.003075 -0.005688  0.009646  0.015581 -0.015476 -0.015496   0.02018   
8  -0.000183 -0.005572  0.012197  0.019955  -0.01754 -0.016243  0.022254   
9        0.0       0.0       0.0       0.0       0.0       0.0       0.0   
10  0.010876  -0.02078  0.002027  0.003308  0.014065 -0.014967 -0.000688   
11       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
12 -0.001828

In [172]:
data = pd.read_csv('/home/sx/MPI_profile/is.A.x_pmu/graph_test_metric.csv', header=[0,1])
data.columns.names = ['metric', 'host']
tempm = data.stack()
# print(data)
# print(tempm)
tempm = (tempm-tempm.mean())/(tempm.std())
metric = tempm.unstack().swaplevel('metric','host',axis=1).stack().unstack()
print(metric)

host      node_0                                                              \
metric  vector_0  vector_1 vector_10 vector_11 vector_12 vector_13 vector_14   
0      -0.168027  0.495453 -0.216318  0.526134  0.185839 -0.242683  0.585657   
1      -0.168027  0.495453 -0.216318  0.526134  0.185839 -0.242683  0.585657   
2      -0.168027  0.495453 -0.216318  0.526134  0.185839 -0.242683  0.585657   
3      -0.168027  0.495453 -0.216318  0.526134  0.185839 -0.242683  0.585657   
4       3.853404 -3.111014 -1.747022 -0.514413 -0.319603  0.290099 -3.028825   
5      -1.045658  0.885863  1.416950  1.470153  4.768908 -1.901976 -0.579354   
6      -0.905371 -0.574789  1.616391 -0.724935 -1.705710  0.938419 -2.272663   
7      -0.211880 -0.553008  1.505161  1.092938  0.839459  2.102369 -2.145791   
8      -0.168027  0.495453 -0.216318  0.526134  0.185839 -0.242683  0.585657   
9       2.439575 -3.414350 -0.359558 -0.083035 -0.328803 -1.189177 -1.555969   
10     -0.168027  0.495453 -0.216318  0.

In [170]:
import torch

input_edge = "/home/sx/MPI_profile/is.A.x_pmu/graph_test_edge"
edge_data = pd.read_csv(input_edge, header=0)
edge_data = edge_data.groupby(['ts_id','src', 'dst']).agg({'commsize': 'sum'}).reset_index()
# print(edge_data)

edge_index = edge_data[['src', 'dst']].values.T.tolist()
edge_weight = edge_data['commsize'].values.tolist()

# print("Edge Index:")
# print(edge_index)

# print("Edge Weight:")
# print(edge_weight)


   ts_id  src  dst  commsize
0    842    3    4         1
1    843    1    2         1
2    843    5    6         1
3    845    4    5         1
4    846    2    3         1
5    846    6    7         1
6    849    0    1         1
Edge Index:
[[3, 1, 5, 4, 2, 6, 0], [4, 2, 6, 5, 3, 7, 1]]
Edge Weight:
[1, 1, 1, 1, 1, 1, 1]
